## Summarize the stock news

I will use yfinance api to get news about a company.
Via yfinance we can extract an URL from the news. Then we can scrape the website. Finally we will ask an LLM to summarize the content of that website.

News is returned as a dictionary. The URL can be found under the keys 'content', 'canonicalUrl' and 'url'.
So it's nested like this: news['content']['canonicalUrl']['url']

At the beginning of the notebook you can see that I was testing the API, what data type it returns, how to extract the information etc.

Then I have defined the acttual code to execute this project. As for the LLM, I use OpenAI gpt-4.1-mini.

In [1]:
import yfinance as yf

In [ ]:

tickers = ["AAPL"]
info_list = []
for ticker in tickers:
    ticker = yf.Ticker(ticker)
    # Get general info about the company
    info_list.append(ticker.info)
    print(f"{ticker} Info: \n {info_list[0]}")

yfinance.Ticker object <AAPL> Info: 
 {'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, m

In [ ]:
news = ticker.news
print("\nRecent News:")
for n in news[:1]:
    print(n)
    print(n['content'])
    print(n['content']['title'])
    n_content = n['content']
    for key,value in n_content.items():
        print(f"key:"{key})
        print(f"value: \n{value}")



Recent News:
{'id': '25dbcf43-934f-3c7c-afa0-55594211df4b', 'content': {'id': '25dbcf43-934f-3c7c-afa0-55594211df4b', 'contentType': 'VIDEO', 'title': 'Micron to cease chip sales to China, bitcoin sinks to 4-month low', 'description': '<p>Market Catalysts host <a data-i13n="cpos:1;pos:1" href="https://www.yahoo.com/author/alexandra-canal/?.tsrc=fin-srch">Allie Canal</a> tracks several of the day\'s top trending stock tickers, including Micron Technology\'s (<a data-i13n="cpos:2;pos:1" href="https://finance.yahoo.com/quote/MU">MU</a>) move to stop supplying chips to data centers in China, bitcoin (<a data-i13n="cpos:3;pos:1" href="https://finance.yahoo.com/quote/BTC-USD">BTC-USD</a>) plunging this week to a four-month low, and Apple (<a data-i13n="cpos:4;pos:1" href="https://finance.yahoo.com/quote/AAPL">AAPL</a>) signing a deal to be the official broadcaster of Formula One (<a data-i13n="cpos:5;pos:1" href="https://finance.yahoo.com/quote/FWONK">FWONK</a>, <a data-i13n="cpos:6;pos:1" 

In [ ]:
ticker = 'AAPL'
ticker = yf.Ticker(ticker)
news = ticker.news
print("\nRecent News:")
for n in news[:2]:
    print(n)
    print(n['content'])
    print(n['content']['title'])
    n_content = n['content']
    for key,value in n_content.items():
        print(f"key: {key}")
        print(f"{value}")


Recent News:
{'id': 'aa6d4ee1-1bb9-3678-93a2-ff1d2e359744', 'content': {'id': 'aa6d4ee1-1bb9-3678-93a2-ff1d2e359744', 'contentType': 'STORY', 'title': 'Apple announces five-year partnership as U.S. broadcast partner for Formula 1', 'description': '', 'summary': 'Apple (AAPL) and Formula 1 announced a five-year partnership that will bring all F1 races exclusively to Apple TV in the United States beginning next year. “The partnership builds on Apple’s deepening relationship with Formula 1 following the global success of Apple Original Films’ adrenaline-fueled blockbuster F1 The Movie, the highest-grossing sports movie of all time. United by a commitment to innovation and fan experience, the partnership sets the stage for Formula 1’s continued growth in th', 'pubDate': '2025-10-18T13:05:12Z', 'displayTime': '2025-10-18T13:05:12Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/tipranks_452/f5079a739fb47b983bae87f28f5b9

In [ ]:
# extracting the website content

from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:4_000]

In [ ]:
ticker = 'AAPL'
ticker = yf.Ticker(ticker)
news = ticker.news
print("\nRecent News:")
for n in news[:1]:
    print(n['content']['title'])
    n_content = n['content']
    url = n_content['canonicalUrl']['url']
    print(url)



Recent News:
Apple announces five-year partnership as U.S. broadcast partner for Formula 1
https://sports.yahoo.com/article/apple-announces-five-partnership-u-130512518.html


In [20]:
fetch_website_contents(url)

"Apple announces five-year partnership as U.S. broadcast partner for Formula 1 - Yahoo Sports\n\nSearch query\nSearch\nNews\nFinance\nSports\nMore\n-1\nManage your account\nHelp\nAdd or switch accounts\nSign out\nNFL\nNCAAF\nFantasy\nMLB\nNBA\nNHL\nWNBA\nGolf\nBetting\nRacing\nTennis\nNCAAB\nWatch\nNCAAW\nDaily Draw\nSoccer\nCombat\nMMA\nBoxing\nWrestling\nBoardroom\nCollectibles\nWomen's Sports\nCollege & High School\nFantasy Football Draft Kit\nCollege Sports\nHorse Racing\nUFL\nCycling\nOlympics\nCricket\nWhat & How To Watch\nNewsletters\nGameChannel\nWatch\nRSS\nJobs\nHelp\nMore News\nBoardroom\nNetwork with Rich Kleiman\nEpisode 7: CC Sabathia\nEpisode 6: Donovan Mitchell\nEpisode 5: Karl-Anthony Towns\nEpisode 4: Mark Cuban\nCover Story: Issa Rae\nCover Story: Aryna Sabalenka\nSee All Boardroom\nExplore Boardroom, where Yahoo and Boardroom Sports cover the business and culture behind the biggest sports stories.\nCombat\nMMA\nBoxing\nWrestling\nThe Ariel Helwani Show\nThe Boys in 

In [21]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# Next is to load the API keys into environment variables
# it ensures that what's in the file with our API keys takes priority

load_dotenv(override=True)

True

In [26]:
system_prompt = """
You are a helpful assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

Title of the article:


"""

In [27]:
def messages_create(website, title):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + title + website}
    ]

In [28]:
def summarize(url, title):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_create(website, title)
    )
    return response.choices[0].message.content

In [29]:
def display_summary(url, title):
    summary = summarize(url, title)
    display(Markdown(summary))

In [ ]:
openai = OpenAI()

tickers = ["AAPL"]
for ticker in tickers:
    ticker = yf.Ticker(ticker)
    news = ticker.news
    for n in news[:1]:
        n_content = n['content']
        title = n['content']['title']
        url = n_content['canonicalUrl']['url']
        display_summary(url, title)

Apple just revved up the streaming game by snagging exclusive U.S. broadcast rights for Formula 1 for the next five years—starting next season, all F1 races will zoom only on Apple TV. This partnership follows the smashing success of "F1 The Movie," the highest-grossing sports flick ever. Expect full coverage of practices, qualifiers, sprints, and grands prix, with some content even free on the Apple TV app. Plus, Apple’s going all-in by boosting F1 content across Apple News, Maps, Music, Fitness+, and their Sports app, turning your iPhone into the ultimate race pit. So if you thought Apple was just about phones and fruit, think again—they’re now your go-to for speed and drama on the track! 🏎️🍏